In [1]:
!nvidia-smi

Mon Aug  2 11:44:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8    34W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
!wget -q https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar xf spark-2.4.8-bin-hadoop2.7.tgz

#!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
#!tar xf spark-3.1.2-bin-hadoop3.2.tgz

In [4]:
!pip install -q findspark

In [5]:
!wget https://repo1.maven.org/maven2/ai/rapids/cudf/0.9.2/cudf-0.9.2-cuda10-1.jar

#!wget https://repo1.maven.org/maven2/ai/rapids/cudf/21.06.1/cudf-21.06.1-cuda11.jar
#downloading RAPIDS ai to run on GPU. It uses GPU DataReader & GPU DataFrame

!wget https://repo1.maven.org/maven2/ai/rapids/xgboost4j_2.x/1.0.0-Beta5/xgboost4j_2.x-1.0.0-Beta5.jar
!wget https://repo1.maven.org/maven2/ai/rapids/xgboost4j-spark_2.x/1.0.0-Beta5/xgboost4j-spark_2.x-1.0.0-Beta5.jar

--2021-08-02 11:44:27--  https://repo1.maven.org/maven2/ai/rapids/cudf/0.9.2/cudf-0.9.2-cuda10-1.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64759406 (62M) [application/java-archive]
Saving to: ‘cudf-0.9.2-cuda10-1.jar’

cudf-0.9.2-cuda10-1 100%[===================>]  61.76M   123MB/s    in 0.5s    

2021-08-02 11:44:28 (123 MB/s) - ‘cudf-0.9.2-cuda10-1.jar’ saved [64759406/64759406]

--2021-08-02 11:44:28--  https://repo1.maven.org/maven2/ai/rapids/xgboost4j_2.x/1.0.0-Beta5/xgboost4j_2.x-1.0.0-Beta5.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 349225621 (333M) [application/java-archive]
Saving to: ‘xgboost4j_2.x-1.0.0-Beta5.jar’

xgboos

In [6]:
!ls

cudf-0.9.2-cuda10-1.jar    spark-2.4.8-bin-hadoop2.7.tgz
sample_data		   xgboost4j_2.x-1.0.0-Beta5.jar
spark-2.4.8-bin-hadoop2.7  xgboost4j-spark_2.x-1.0.0-Beta5.jar


In [7]:
!pwd

/content


In [8]:
import os
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64"
#os.environ["SPARK_HOME"]="/content/spark-3.1.2-bin-hadoop3.2"
os.environ['SPARK_HOME']="/content/spark-2.4.8-bin-hadoop2.7"

In [9]:
#When we run a Spark job, we need to run SparkSubmit and therein we need to package all our dependencies together & submit
#Here we are packaging all the 3 jar files that we downloaded in cell [9] and submitting
#os.environ["PYSPARK_SUBMIT_ARGS"]="--jars /content/cudf-21.06.1-cuda11.jar,/content/xgboost4j_2.x-1.0.0-Beta5.jar,/content/xgboost4j-spark_2.x-1.0.0-Beta5.jar pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--jars /content/cudf-0.9.2-cuda10-1.jar,/content/xgboost4j_2.x-1.0.0-Beta5.jar,/content/xgboost4j-spark_2.x-1.0.0-Beta5.jar pyspark-shell"

#os.environ["PYSPARK_SUBMIT_ARGS"]="--master local[*]"
#os.environ["PYSPARK_SUBMIT_ARGS"]="--pyspark-shell"
#Due to a change in python/pyspark/java_gateway.py , it is now required that PYSPARK_SUBMIT_ARGS includes pyspark-shell if a 
#PYSPARK_SUBMIT_ARGS variable is set by a user.

In [10]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [11]:
spark.conf.set("spark.executor.memory","2g") #This is not allowed in Spark 3.x onwards
spark.conf.set("spark.driver.memory","2g")  #This is not allowed in Spark 3.x onwards
spark.sparkContext.addPyFile("/content/xgboost4j-spark_2.x-1.0.0-Beta5.jar") #Adding the Python bindings for XGBoostSpark to the PyFile.

In [12]:
from ml.dmlc.xgboost4j.scala.spark import XGBoostClassificationModel, XGBoostClassifier
from ml.dmlc.xgboost4j.scala.spark.rapids import GpuDataReader
#The above packages are imported from xgboost4j-spark_2.x-1.0.0-Beta5
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

import numpy as np
import pandas as pd

In [13]:
from sklearn.datasets import fetch_openml
covtyp=fetch_openml(name='covertype',version=4)
covtyp.data.shape

(581012, 54)

In [14]:
np.unique(covtyp.target)

array(['1', '2', '3', '4', '5', '6', '7'], dtype=object)

In [15]:
!nvidia-smi

Mon Aug  2 11:45:04 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    34W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
cov_df = pd.DataFrame(data= np.c_[covtyp['data'], covtyp['target']],
                     columns= covtyp['feature_names'] + ['target'])

In [17]:
cov_df.memory_usage(index=True).sum() #the data is still in pandas dataframe

255645408

In [18]:
cov_df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,target
0,2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5


In [19]:
print ("Rows     : " ,cov_df.shape[0])
print ("Columns  : " ,cov_df.shape[1])

Rows     :  581012
Columns  :  55


In [20]:
cov_df.target.value_counts()

2    283301
1    211840
3     35754
7     20510
6     17367
5      9493
4      2747
Name: target, dtype: int64

In [21]:
cov_df.dtypes

Elevation                             object
Aspect                                object
Slope                                 object
Horizontal_Distance_To_Hydrology      object
Vertical_Distance_To_Hydrology        object
Horizontal_Distance_To_Roadways       object
Hillshade_9am                         object
Hillshade_Noon                        object
Hillshade_3pm                         object
Horizontal_Distance_To_Fire_Points    object
Wilderness_Area1                      object
Wilderness_Area2                      object
Wilderness_Area3                      object
Wilderness_Area4                      object
Soil_Type1                            object
Soil_Type2                            object
Soil_Type3                            object
Soil_Type4                            object
Soil_Type5                            object
Soil_Type6                            object
Soil_Type7                            object
Soil_Type8                            object
Soil_Type9

In [22]:
for cols in cov_df.columns:
  cov_df[cols] = pd.to_numeric(cov_df[cols])

In [23]:
cov_df.dtypes

Elevation                             float64
Aspect                                float64
Slope                                 float64
Horizontal_Distance_To_Hydrology      float64
Vertical_Distance_To_Hydrology        float64
Horizontal_Distance_To_Roadways       float64
Hillshade_9am                         float64
Hillshade_Noon                        float64
Hillshade_3pm                         float64
Horizontal_Distance_To_Fire_Points    float64
Wilderness_Area1                      float64
Wilderness_Area2                      float64
Wilderness_Area3                      float64
Wilderness_Area4                      float64
Soil_Type1                            float64
Soil_Type2                            float64
Soil_Type3                            float64
Soil_Type4                            float64
Soil_Type5                            float64
Soil_Type6                            float64
Soil_Type7                            float64
Soil_Type8                        

In [24]:
cov_df['target'] = cov_df['target']-1
np.unique(cov_df.target)

array([0, 1, 2, 3, 4, 5, 6])

In [25]:
train_df=cov_df.sample(frac=0.8,random_state=10)
test_df=cov_df.drop(train_df.index)
#Creating a train-test split in ration of 80:20 for training:testing & random_state is set to 10 for reproducability

In [26]:
train_df.to_parquet(path='covtype_train.parquet',compression='snappy',index=False)
test_df.to_parquet(path='covtype_test.parquet',compression='snappy',index=False)
#saving the dataset after cleansing & pre-processing. This is a standard approach in Big-Data ML

In [27]:
!ls

covtype_test.parquet	 spark-2.4.8-bin-hadoop2.7
covtype_train.parquet	 spark-2.4.8-bin-hadoop2.7.tgz
cudf-0.9.2-cuda10-1.jar  xgboost4j_2.x-1.0.0-Beta5.jar
sample_data		 xgboost4j-spark_2.x-1.0.0-Beta5.jar


In [28]:
import pyarrow.parquet as pq
pq.read_table('covtype_train.parquet')

pyarrow.Table
Elevation: double
Aspect: double
Slope: double
Horizontal_Distance_To_Hydrology: double
Vertical_Distance_To_Hydrology: double
Horizontal_Distance_To_Roadways: double
Hillshade_9am: double
Hillshade_Noon: double
Hillshade_3pm: double
Horizontal_Distance_To_Fire_Points: double
Wilderness_Area1: double
Wilderness_Area2: double
Wilderness_Area3: double
Wilderness_Area4: double
Soil_Type1: double
Soil_Type2: double
Soil_Type3: double
Soil_Type4: double
Soil_Type5: double
Soil_Type6: double
Soil_Type7: double
Soil_Type8: double
Soil_Type9: double
Soil_Type10: double
Soil_Type11: double
Soil_Type12: double
Soil_Type13: double
Soil_Type14: double
Soil_Type15: double
Soil_Type16: double
Soil_Type17: double
Soil_Type18: double
Soil_Type19: double
Soil_Type20: double
Soil_Type21: double
Soil_Type22: double
Soil_Type23: double
Soil_Type24: double
Soil_Type25: double
Soil_Type26: double
Soil_Type27: double
Soil_Type28: double
Soil_Type29: double
Soil_Type30: double
Soil_Type31: doubl

In [36]:
#train_data = GpuDataReader(spark).format('parquet').load('covtype_train.parquet')
train_data = GpuDataReader(spark).parquet('covtype_train.parquet')
test_data = GpuDataReader(spark).parquet('covtype_test.parquet')
#Pulling data from physical memory (stored in the form of parquet) to GPU memory

In [37]:
train_data.schema

<bound method GpuDataset.schema of <ml.dmlc.xgboost4j.scala.spark.rapids.GpuDataset object at 0x7f413ec5f210>>

In [38]:
pq_file=pq.read_table('covtype_train.parquet')
pq_file.column_names

['Elevation',
 'Aspect',
 'Slope',
 'Horizontal_Distance_To_Hydrology',
 'Vertical_Distance_To_Hydrology',
 'Horizontal_Distance_To_Roadways',
 'Hillshade_9am',
 'Hillshade_Noon',
 'Hillshade_3pm',
 'Horizontal_Distance_To_Fire_Points',
 'Wilderness_Area1',
 'Wilderness_Area2',
 'Wilderness_Area3',
 'Wilderness_Area4',
 'Soil_Type1',
 'Soil_Type2',
 'Soil_Type3',
 'Soil_Type4',
 'Soil_Type5',
 'Soil_Type6',
 'Soil_Type7',
 'Soil_Type8',
 'Soil_Type9',
 'Soil_Type10',
 'Soil_Type11',
 'Soil_Type12',
 'Soil_Type13',
 'Soil_Type14',
 'Soil_Type15',
 'Soil_Type16',
 'Soil_Type17',
 'Soil_Type18',
 'Soil_Type19',
 'Soil_Type20',
 'Soil_Type21',
 'Soil_Type22',
 'Soil_Type23',
 'Soil_Type24',
 'Soil_Type25',
 'Soil_Type26',
 'Soil_Type27',
 'Soil_Type28',
 'Soil_Type29',
 'Soil_Type30',
 'Soil_Type31',
 'Soil_Type32',
 'Soil_Type33',
 'Soil_Type34',
 'Soil_Type35',
 'Soil_Type36',
 'Soil_Type37',
 'Soil_Type38',
 'Soil_Type39',
 'Soil_Type40',
 'target']

In [39]:
label="target"
features=[x for x in pq_file.column_names if x!=label]

In [40]:
features

['Elevation',
 'Aspect',
 'Slope',
 'Horizontal_Distance_To_Hydrology',
 'Vertical_Distance_To_Hydrology',
 'Horizontal_Distance_To_Roadways',
 'Hillshade_9am',
 'Hillshade_Noon',
 'Hillshade_3pm',
 'Horizontal_Distance_To_Fire_Points',
 'Wilderness_Area1',
 'Wilderness_Area2',
 'Wilderness_Area3',
 'Wilderness_Area4',
 'Soil_Type1',
 'Soil_Type2',
 'Soil_Type3',
 'Soil_Type4',
 'Soil_Type5',
 'Soil_Type6',
 'Soil_Type7',
 'Soil_Type8',
 'Soil_Type9',
 'Soil_Type10',
 'Soil_Type11',
 'Soil_Type12',
 'Soil_Type13',
 'Soil_Type14',
 'Soil_Type15',
 'Soil_Type16',
 'Soil_Type17',
 'Soil_Type18',
 'Soil_Type19',
 'Soil_Type20',
 'Soil_Type21',
 'Soil_Type22',
 'Soil_Type23',
 'Soil_Type24',
 'Soil_Type25',
 'Soil_Type26',
 'Soil_Type27',
 'Soil_Type28',
 'Soil_Type29',
 'Soil_Type30',
 'Soil_Type31',
 'Soil_Type32',
 'Soil_Type33',
 'Soil_Type34',
 'Soil_Type35',
 'Soil_Type36',
 'Soil_Type37',
 'Soil_Type38',
 'Soil_Type39',
 'Soil_Type40']

In [45]:
import time
params={'eta':0.1,
        'gamma':0.1,
        'missing':0.0,
        'treeMethod':'gpu_hist',
        'maxDepth':8,
        'growPolicy':'depthwise',
        'lambda_':1.0,
        'subsample':1.0,
        'numRound':1000,
        'numWorkers':1,
        'verbosity':1
        }
#classifier = XGBoostClassifier(**params).setLabelCol(label).setFeaturesCols(features)
classifier = XGBoostClassifier(**params).setLabelCol(label).setFeaturesCols(features)

#### XGBoost by default provides lots of mechanisms for distribution like Kubernetes, Dask, YARN, as well as cloud providers. However, here we use Spark ####

In [46]:
start_time=time.time()
model= classifier.fit(train_data)
print("GPU Training Time is: %seconds "%(str(time.time()-start_time)))

Py4JJavaError: ignored

In [ ]:
!nvidia-smi

In [ ]:
model.write().overwrite().save('/content/model/')
#saving the model for further use. In reality we deploy this model in production on a spark environment with no need for re-training

In [ ]:
!ls

In [ ]:
loaded_model = XGBoostClassificationModel().load('/content/model/')

In [ ]:
result = loaded_model.transform(test_data)

In [ ]:
result.show()

In [ ]:
MulticlassClassificationEvaluator().setLabelCol(label).evaluate(result)